# Punto 1

In [238]:
from pyomo.environ import ConcreteModel, RangeSet, \
    Param, Var, Binary, \
    Objective, maximize, Constraint
from matplotlib import pyplot as plt
from pyomo.opt import SolverFactory

## Parte A

In [239]:
tasks_num = 11
max_points = 52

model = ConcreteModel()

# sets
model.T = RangeSet(1, tasks_num) # 1, 2, ..., 11

# params
ph = [5, 3, 13, 1, 21, 2, 2, 5, 8, 13, 21]
priority = [7, 5, 6, 3, 1, 4, 6, 6, 2, 7, 6]
dict_ph = {i+1: ph[i] for i in range(tasks_num)}
dict_priority = {i+1: priority[i] for i in range(tasks_num)}

model.h_points = Param(model.T, initialize=dict_ph)
model.priority = Param(model.T, initialize=dict_priority)
model.max_points = Param(initialize=max_points)

# dec. variable
model.X = Var(model.T, within=Binary)

Recordemos del informe que lo que debemos hacer es máximizar la prioridad de las tareas que se asignaron a los programadores.

$$ Z = \max \sum_{i \in T} w_i \times X_i $$

In [240]:
# obj. function
def obj_func(model):
    return sum(model.priority[i] * model.X[i] for i in model.T)

model.obj = Objective(rule=obj_func, sense=maximize)

Recordemos que hay una restricción sobre los puntos de historias de usuario. En este caso el maximo de punto de las tareas asignadas al equipo son 52 puntos.

$$ \sum_{i \in T} X_i \times p_i \leq H^{*} $$

In [241]:
if hasattr(model, "max_history_points"):
    model.del_component(model.max_history_points)

def constraint_max_history_points(model):
    return sum(model.X[i] * model.h_points[i] for i in model.T) <= model.max_points

model.max_history_points = Constraint(rule=constraint_max_history_points)

In [242]:
solver = SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 46.0, 'Upper bound': 46.0, 'Number of objectives': 1, 'Number of constraints': 1, 'Number of variables': 11, 'Number of nonzeros': 11, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.03463602066040039}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [244]:
for i in model.T:
    if model.X[i].value == 1:
        print(f"La tarea {i} se asigno al equipo de desarrolladores")

print()
total = sum(model.X[i].value * model.h_points[i] for i in model.T)

print(f"El total de puntos de historia utilizados son {total}")

La tarea 1 se asigno al equipo de desarrolladores
La tarea 2 se asigno al equipo de desarrolladores
La tarea 3 se asigno al equipo de desarrolladores
La tarea 4 se asigno al equipo de desarrolladores
La tarea 6 se asigno al equipo de desarrolladores
La tarea 7 se asigno al equipo de desarrolladores
La tarea 8 se asigno al equipo de desarrolladores
La tarea 9 se asigno al equipo de desarrolladores
La tarea 10 se asigno al equipo de desarrolladores

El total de puntos de historia utilizados son 52.0


# Parte B

En esta parte se nos pide modificar el modelo para asignar las tareas a cada desarrollador maximizando la prioridad de la mismas, sujeto a que cada uno tiene un maximo de puntos de historia.

In [245]:
tasks = 11
developers = 4
max_history_points_per_dev = 13

model = ConcreteModel()

model.T = RangeSet(1, tasks)
model.D = RangeSet(1, developers)

# parameters 
ph = [5, 3, 13, 1, 21, 2, 2, 5, 8, 13, 21]
priority = [7, 5, 6, 3, 1, 4, 6, 6, 2, 7, 6]
dict_ph = {i+1: ph[i] for i in range(tasks_num)}
dict_priority = {i+1: priority[i] for i in range(tasks_num)}

model.h_points = Param(model.T, initialize=dict_ph)
model.priority = Param(model.T, initialize=dict_priority)
model.max_h_points = Param(initialize=max_history_points_per_dev)

# decision variable
model.X = Var(model.D, model.T, within=Binary)

La función objetivo es la siguiente.

$$ Z = \max \sum_{j \in D} \sum_{i \in T} X_{i, j} \times w_i $$


In [246]:
def obj_func(model):
    return sum(model.X[i, j] * model.priority[j] for j in model.T for i in model.D)

model.obj = Objective(rule=obj_func, sense=maximize)

Sobre las restricciones tenemos que las tareas asignadas a un desarrollador no pueden exceder el límite de 13 y que una tarea no puede ser asignada a dos desarrolladores.

$$
\sum_{j \in T}  p_j \times X_{i, j} \leq d_i, \forall i \in D
$$

$$
\sum_{i \in D}  X_{i, j} \leq 1, \forall j \in T
$$

In [247]:
def max_history_points_per_dev_constr(model, i):
    return sum(model.X[i, j] * model.h_points[j] for j in model.T) <= model.max_h_points

model.history_points_constr = Constraint(model.D, rule=max_history_points_per_dev_constr)

def assign_one_task_per_dev_constr(model, j):
    return sum(model.X[i,j] for i in model.D) <= 1

model.one_task_constr = Constraint(model.T, rule=assign_one_task_per_dev_constr)

In [248]:
solver = SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 46.0, 'Upper bound': 46.0, 'Number of objectives': 1, 'Number of constraints': 15, 'Number of variables': 44, 'Number of nonzeros': 88, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '5', 'Number of created subproblems': '5'}}, 'Error rc': 0, 'Time': 0.04733085632324219}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [249]:
for i in model.D:
    for j in model.T:
        if model.X[i, j].value == 1:
            print(f"Desarrollador {i} tiene tarea {j}")
print()

total = sum(model.X[i, j].value * model.h_points[j] for j in model.T for i in model.D)
print(f"Número total de puntos de historia: {total}")

Desarrollador 1 tiene tarea 1
Desarrollador 1 tiene tarea 2
Desarrollador 1 tiene tarea 8
Desarrollador 2 tiene tarea 10
Desarrollador 3 tiene tarea 3
Desarrollador 4 tiene tarea 4
Desarrollador 4 tiene tarea 6
Desarrollador 4 tiene tarea 7
Desarrollador 4 tiene tarea 9

Número total de puntos de historia: 52.0


# Punto 3

En este punto se nos pide maximizar el valor de los recursos se llevan en la flota de 3 aviones, respetando volumen y peso maximo que un avión puede transportar.

In [250]:
flights = 3
resources = 5

model = ConcreteModel()

model.F = RangeSet(1, flights)
model.R = RangeSet(1, resources)

# constraints
# resources
resource_weight = [15, 5, 20, 18, 10]
resource_volume = [8, 2, 10, 12, 6]
resource_value = [50, 100, 120, 60, 40]

dict_rw = {i+1: resource_weight[i] for i in range(resources)}
dict_rv = {i+1: resource_volume[i] for i in range(resources)}
dict_rval = {i+1: resource_value[i] for i in range(resources)}

# flights
flight_wcapacity = [30, 40, 50]
flight_vcapacity = [25, 30, 35]

dict_fw = {i+1: flight_wcapacity[i] for i in range(flights)} # Weight 
dict_fv = {i+1: flight_vcapacity[i] for i in range(flights)} # Volume

# define constraints in the model
model.RW = Param(model.R, initialize=dict_rw)
model.RV = Param(model.R, initialize=dict_rv)
model.RVal = Param(model.R, initialize=dict_rval)

model.FW = Param(model.F, initialize=dict_fw)
model.FV = Param(model.F, initialize=dict_fv)

# define decision variable
model.X = Var(model.F, model.R, within=Binary)

La función objetivo es la siguiente:

$$
    Z = \max \sum_{i \in F} \sum_{j \in R} X_{i, j} \times p_j
$$

In [251]:
def obj_func(model):
    return sum(model.X[i, j] * model.RVal[j] for j in model.R for i in model.F)

model.obj = Objective(rule=obj_func, sense=maximize)

Adicionalmente, tenemos que cumplir con las restricciones de capacidad maxima del avion, volumen y peso.

In [252]:
def max_weight_constr(model, i):
    return sum(model.X[i, j] * model.RW[j] for j in model.R) <= model.FW[i]

model.max_weight = Constraint(model.F, rule=max_weight_constr)

def max_volume_constr(model, i):
    return sum(model.X[i, j] * model.RV[j] for j in model.R) <= model.FV[i]

model.max_volumen = Constraint(model.F, rule=max_volume_constr)

Además, solo podemos enviar un recurso entre toda la flota. Es decir, no se van a mandar 2 instancias de alimentos básicos.

In [253]:
def one_resource_constr(model, j):
    return sum(model.X[i, j] for i in model.F) <= 1

model.one_resource = Constraint(model.R, rule=one_resource_constr)

Finalmente, contamos con otras dos restricciones. La primera nos dice que las medicinas no van en el avión 1. La segunda es que el agua no puede ir en el mismo avión que los equipos medicos.

In [254]:
def resources_constr(model, i):
    return model.X[i, 3] + model.X[i, 4] <= 1
model.resources_constr = Constraint(model.F, rule=resources_constr)

model.exclusion_constraint = Constraint(expr=model.X[1,2] == 0)

In [255]:
solver = SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 370.0, 'Upper bound': 370.0, 'Number of objectives': 1, 'Number of constraints': 15, 'Number of variables': 15, 'Number of nonzeros': 52, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '3', 'Number of created subproblems': '3'}}, 'Error rc': 0, 'Time': 0.04981565475463867}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [258]:
for i in model.F:
    for j in model.R:
        if model.X[i, j].value == 1:
            print(f"El avion {i} lleva el recurso {j}")
print()

total = sum(model.X[i, j].value * model.RVal[j] for j in model.R for i in model.F)
print(f"El valor total de los recursos que llevo es {total}")

El avion 1 lleva el recurso 1
El avion 2 lleva el recurso 2
El avion 2 lleva el recurso 3
El avion 2 lleva el recurso 5
El avion 3 lleva el recurso 4

El valor total de los recursos que llevo es 370.0
